In [53]:
import tools
import importlib
import memory
importlib.reload(tools)
importlib.reload(memory)

from tools import network_tools
from tools import firewall_tools

In [70]:
alerts = await network_tools.get_alerts(time_window=5)
alerts

{'security_events': {'events_by_type': [{'category': 'Potentially Bad Traffic',
    'dest_ip': '172.20.0.10',
    'dest_port': 3306,
    'flow_id': 549109394661489,
    'payload': None,
    'proto': 'TCP',
    'severity': 2,
    'signature': 'ET SCAN Suspicious inbound to mySQL port 3306',
    'src_ip': '192.168.100.2',
    'src_port': 44748,
    'timestamp': '2025-08-08T17:56:17.193385+0200'},
   {'category': 'Attempted Information Leak',
    'dest_ip': '172.20.0.10',
    'dest_port': 8080,
    'flow_id': 407927618563837,
    'payload': None,
    'proto': 'TCP',
    'severity': 2,
    'signature': 'POSSBL PORT SCAN (NMAP -sS)',
    'src_ip': '192.168.100.2',
    'src_port': 44748,
    'timestamp': '2025-08-08T17:56:17.291586+0200'},
   {'category': 'Attempted Information Leak',
    'dest_ip': '172.20.0.10',
    'dest_port': 389,
    'flow_id': 418132985540249,
    'payload': None,
    'proto': 'TCP',
    'severity': 2,
    'signature': 'POSSBL PORT SCAN (NMAP -sS)',
    'src_ip': '192

In [64]:
network_tools.get_docker_containers()

{'honeypot_config': [{'id': '06dbe9c34a92',
   'name': 'cve-2015-5254-activemq-1',
   'image': 'vulhub/activemq:5.11.1',
   'status': 'running',
   'created': '2025-08-08T10:36:22.911536072Z',
   'ports': ['61616/tcp', '8161/tcp'],
   'ip_address': '172.20.0.6'},
  {'id': '9d508304362e',
   'name': 'cve-2021-22205-gitlab-1-proxy',
   'image': 'nginx:alpine',
   'status': 'running',
   'created': '2025-08-08T10:36:21.971744106Z',
   'ports': ['80/tcp'],
   'ip_address': '172.20.0.10'},
  {'id': '2ee7e207ca0a',
   'name': 's2-057-struts2-1',
   'image': 'vulhub/struts2:2.3.34-showcase',
   'status': 'running',
   'created': '2025-08-08T10:36:21.608542529Z',
   'ports': ['8080/tcp'],
   'ip_address': '172.20.0.2'},
  {'id': '640353ced621',
   'name': 'cve-2014-6271-web-1',
   'image': 'vulhub/bash:4.3.0-with-httpd',
   'status': 'running',
   'created': '2025-08-06T07:48:02.488925808Z',
   'ports': ['80/tcp'],
   'ip_address': '172.20.0.7'}]}

In [73]:
print(network_tools.check_services_health())
rules = await firewall_tools.get_firewall_rules()
print(rules['firewall_config']['data']['rules'])

{'firewall_status': 'up', 'suricata_status': 'up'}
Chain FORWARD (policy DROP)
num  target     prot opt source               destination         
1    ACCEPT     all  --  0.0.0.0/0            0.0.0.0/0            state RELATED,ESTABLISHED
2    ACCEPT     icmp --  0.0.0.0/0            0.0.0.0/0           
3    ACCEPT     all  --  172.20.0.0/24        172.20.0.0/24       
4    DROP       all  --  192.168.100.0/24     172.20.0.0/24       
5    DROP       all  --  172.20.0.0/24        192.168.100.0/24    
6    LOG        all  --  0.0.0.0/0            0.0.0.0/0            LOG flags 0 level 4 prefix "FIREWALL-DROP: "



In [72]:
await firewall_tools.remove_firewall_rule([1])

INFO:tools.firewall_tools:Removing firewall rules: [1]


{'rules_removed_current_epoch': ['REMOVED rule #1:    ACCEPT     tcp  --  192.168.100.0/24     172.20.0.10']}

In [65]:
honeypot_ip = '172.20.0.10' 
attacker_ip = '192.168.100.2/24'

await firewall_tools.add_allow_rule(source_ip=attacker_ip, dest_ip=honeypot_ip)
await firewall_tools.add_allow_rule(source_ip=honeypot_ip, dest_ip=attacker_ip)

INFO:tools.firewall_tools:Adding allow rule: ALLOW 192.168.100.2/24 -> 172.20.0.10 (tcp)
INFO:tools.firewall_tools:Adding allow rule: ALLOW 172.20.0.10 -> 192.168.100.2/24 (tcp)


{'rules_added_current_epoch': 'ALLOW 172.20.0.10 -> 192.168.100.2/24 (tcp)'}